## Blacklisted Topic Detection (LLM Post Proc - Version 1)

### Latent Dirichlet Allocation (LDA) for Topic Modelling

- In the first version, we train our LDA model on a general corpus of 20k news articles belonging to various topic categories.
- We will use this model to then blacklist responses of a black box LLM which are Political or Religious in nature.

In [1]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_data = fetch_20newsgroups()

In [2]:
print(list(newsgroups_data.target_names))

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [3]:
print(newsgroups_data.filenames.shape, newsgroups_data.target.shape)

(11314,) (11314,)


In [4]:
import ssl

# Create an SSL context without certificate verification
ssl._create_default_https_context = ssl._create_unverified_context
import nltk
import pickle
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/arnav/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

2. Preprocessing
- Stemming and Lemmatizing the Data

In [5]:
stemmer = SnowballStemmer("english")
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [6]:
processed_docs = []

for doc in newsgroups_data.data:
    processed_docs.append(preprocess(doc))

In [7]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

In [8]:
'''
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)

In [9]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

- Training the Model:

In [10]:
# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
# TODO
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 8, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [11]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.008*"presid" + 0.006*"clinton" + 0.004*"netcom" + 0.004*"homosexu" + 0.004*"virginia" + 0.003*"money" + 0.003*"run" + 0.003*"pitch" + 0.003*"talk" + 0.003*"consid"


Topic: 1 
Words: 0.009*"govern" + 0.007*"armenian" + 0.006*"israel" + 0.005*"kill" + 0.005*"isra" + 0.004*"american" + 0.004*"turkish" + 0.004*"weapon" + 0.004*"jew" + 0.004*"live"


Topic: 2 
Words: 0.017*"game" + 0.015*"team" + 0.011*"play" + 0.009*"player" + 0.008*"hockey" + 0.006*"season" + 0.005*"leagu" + 0.005*"canada" + 0.005*"score" + 0.004*"andrew"


Topic: 3 
Words: 0.013*"window" + 0.011*"card" + 0.007*"driver" + 0.007*"drive" + 0.006*"sale" + 0.005*"scsi" + 0.005*"control" + 0.005*"disk" + 0.005*"appl" + 0.005*"price"


Topic: 4 
Words: 0.014*"file" + 0.009*"program" + 0.007*"window" + 0.007*"encrypt" + 0.006*"chip" + 0.006*"imag" + 0.006*"data" + 0.006*"avail" + 0.005*"code" + 0.004*"version"


Topic: 5 
Words: 0.012*"space" + 0.009*"nasa" + 0.005*"scienc" + 0.005*"orbit" + 0.005*"research" 

In [12]:
# Create a dictionary of the identified topics and their contents
topics_dict = {
    0: 'Politics',
    1: 'Geopolitics and Violence',
    2: 'Sports',
    3: 'Ads and Sales',
    4: 'Technology',
    5: 'Space and Science',
    6: 'Automobiles',
    7: 'Religion'
}


- Dump the model to a pkl file

In [13]:
file_path = 'Blacklist_LDA.pkl'

with open(file_path, 'wb') as file:
    pickle.dump(lda_model, file)

- Save the Dictionary also

In [14]:
dict_file_path = 'Blacklist_LDA_Dict.pkl'
dictionary.save(dict_file_path)